## 증권리포트 크롤링

- 매일경제증권센터 증권리포트 항목 크롤링
- 검색어, 검색할 maxPage 설정

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd

#증권리포트 제목, 내용, url을 분리, 저장하는 함수
def get_report_inf(url):
    #headers = {'Accept-Encoding': 'deflate'}
    #res = requests.get(url=url, headers=headers)
    res = requests.get(url=url)
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')#, from_encoding="euc-kr"
    title = soup.find('p', {'class':'tit'}).text
    text = soup.find('div', {'id':'article_main'}).text
    report_data = {}
    report_data['title'] = title.replace("\r"," ").replace("\n"," ").replace("\t"," ").strip()  # 기사 제목
    report_data['text'] = text.replace("\n", " ").replace("\r", " ").replace("<br>", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
    report_data['url'] = url  # 기사 링크
    
    return report_data

In [2]:
#증권리포트 크롤링 data를 반환하는 함수

def get_report_list(page=1  # 뉴스 리스트 페이지
                      ):
    
    
    url='https://www.thebell.co.kr/free/content/article.asp?page={page}&svccode=00'
    
    res = requests.get(url.format(page=page))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')
    #print(soup)
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출    
    table = soup.find('div', {'class':'listBox'})
    alinks = table.findAll('a')
    
    report_list = []
    
    for alink in alinks:
        alink['href'] = 'https://www.thebell.co.kr/free/content/' + alink['href']
        report_url = alink['href']
        report_list.append(get_report_inf(report_url))
    
    return report_list

In [3]:
#증권리포트 크롤링 실행
articles = []

for num in range(1, 6):
    articles.extend(get_report_list(page=num))

print("Total articles: {:,}".format(len(articles)))
#print("="*20, "Article sample", "="*20)
#print(articles[0])

Total articles: 50


In [4]:
df = pd.DataFrame(articles)
df[['title']].head(10)

title
0  코람코신탁, '씨티은행 다동사옥' 몸값 올리기 시동           내년 5월 리...
1  무궁화신탁, 신림백화점 인수자에 '최후통첩'           19일까지 '이행최고...
2  대우건설, 파가니카CC 새 우협에 '스트라이커캐피탈'           8월말 MO...
3  화성산업, '일감 가뭄'에 역성장 지속           [건설리포트]작년부터 수주...
4  [포스코그룹 사업구조 개편]1년만에 '1.2조' 부실사업 정리했다예년대비 손상차손 ...
5  사우디 정유시설 가동중단, 국내 정유업체 영향은           유가상승 불가피,...
6  송승봉 현대엘리베이터 신임 대표 최대 과제는           16년만에 엔지니어 ...
7  다시 주목받는 수소차…현대차 넥쏘 생산량은           보조금 탓에 생산량 확...
8  '이륙 앞둔' 플라이강원…中 노선에 쏠리는 눈           인구 밀도·소득 수...
9  사업구도 정리 마무리 들어간 SK디스커버리           [지배구조 분석]SK건...

## Text preprocessing

- 형태소 분류작업 (정규화, 어근화작업도 수행)

In [5]:
from konlpy.tag import Okt
from collections import Counter
# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)


#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set([])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
#stopwords.add(searchword)  #검색어 자체가 키워드에 포함되지 않게한다


for i, article in enumerate(articles):
        
    words = tagger.pos(article['text'], norm=True, stem=True) #정규화, 어근화도 진행
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    


vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    # 기사 내 서로 다른 단어 개수가 5개 이상인 기사만 사용    
    if len(words) > 5:
        new_words_list.append((words, article))

C:\ProgramData\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [6]:
print(new_words_list[0][0])

['기사', '더벨', '유료', '페이지', '기사', '코람코', '자산', '신탁', '서울', '중구', '다동', '위치', '한국', '씨티', '은행', '사옥', '밸류애드', 'Value', 'add', '본격', '사옥', '기업', '건물', '구분', '소유', '그동안', '모델링', '추진', '어려움', '코람코', '자산', '신탁', '부동산', '매입', '기업', '설득', '건물', '가치', '부동산', '업계', '코람코', '자산', '신탁', '최근', '기업', '협의', '병행', '씨티', '은행', '사옥', '모델링', '준비', '코람코', '자산', '신탁', '관계자', '씨티', '은행', '이전', '시기', '고려', '내년', '모델링', '공사', '시작', '예정', '공사', '개월', '동안', '진행', '계획', '정도', '완공', '전망', '설명', '씨티', '은행', '사옥', '지하', '지상', '면적', '준공', '건물', '한국', '부동산', '신탁', '기업', '구분', '소유', '한미', '은행', '한국', '부동산', '신탁', '전부', '매입', '건물', '사옥', '활용', '미국', '씨티', '그룹', '한미', '은행', '인수', '씨티', '은행', '사옥', '씨티', '은행', '본점', '서울', '여의도', '국제', '금융', '센터', 'IFC', '위해', '사옥', '처분', '스턴', '투자', '운용', '우선', '협상', '대상자', '선정', '매각', '눈앞', '최종', '결렬', '당시', '스턴', '투자', '운용', '국토교통부', '부동산', '투자', '회사', '리츠', '영업', '신청', '정도', '적극', '사옥', '구분', '소유', '건물', '발목', '촬영', '이로', '코람코', '자산', '신탁', '올해', '리츠', '활용', '사옥', '인수', '행보', '관심', '집중'

## Build term-document matrix

In [7]:
#input - words_list, vocab
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


print('기사 갯수 : ' ,len(words_list))
print('vocab 갯수 : ' ,len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

기사 갯수 :  50
vocab 갯수 :  3385
Term document matrix shape: (50, 3385)
  (0, 3355)	0.024671917305329062
  (0, 3351)	0.04926392760534188
  (0, 3341)	0.03568253409686559
  (0, 3327)	0.05189125701696173
  (0, 3283)	0.0633458815101205
  (0, 3281)	0.025945628508480863
  (0, 3253)	0.03167294075506025
  (0, 3251)	0.019556941096653454
  (0, 3238)	0.021724796362719845
  (0, 3217)	0.020218316261901476
  (0, 3205)	0.0633458815101205
  (0, 3200)	0.05796401606128837
  (0, 3151)	0.03502320378817095
  (0, 3116)	0.016421309201780625
  (0, 3108)	0.008262764953057989
  (0, 3039)	0.07115817126245726
  (0, 3031)	0.02259536547537016
  (0, 2967)	0.4202784454580514
  (0, 2961)	0.03502320378817095
  (0, 2898)	0.05352380114529838
  (0, 2885)	0.02356857929501218
  (0, 2874)	0.012647221083029757
  (0, 2871)	0.03167294075506025
  (0, 2836)	0.03502320378817095
  (0, 2793)	0.017340340232748077
  :	:
  (49, 642)	0.0243852210189128
  (49, 635)	0.042241329797293996
  (49, 627)	0.02034448372528149
  (49, 623)	0.0272521694

## Topic modeling

In [8]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF
 
K = 7  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (50, 7)
H shape: (7, 3385)


In [9]:
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')        
    print("\n", "="*50)

0th topic
펀드 투자 출자 기업 운용 WWG 유니온 미래에셋 결성 베트남 메콩 성장 블라인드 해외 GP 산업 회수 조합 프로젝트 포트폴리오 
1th topic
그룹 LG 나이스 카드 케이에스 BC 결제 인수 VAN 사업 매물 사장 전자 디스플레이 PG 유플러스 WM 실사 시스 KKR 
2th topic
SK 케미칼 하이닉스 테크놀로지 CEO 에이 사장 이노베이션 화학 TSMC 디스커버리 입장 LG 콘텐츠 웨이브 보증 채무 분할 부회장 만남 
3th topic
초록 드라마 제작 매출 제작사 넷플릭스 판권 작가 방영 OTT 녹십자 올해 현금 면세점 상반기 산업 콘텐츠 알함브라 사업 실적 
4th topic
금리 발행 미래에셋 자산운용 민평 수요 리츠 배당 회사채 롯데 bp 고유 RBC 모채 스틸 비율 오버 채권 수준 규모 
5th topic
저축은행 웰컴 인수 캐피탈 감독 스트라이커 스마트 신탁 회생 뱅크 당국 금융 주주 씨티 은행 사옥 코람코 신림 백화점 필리핀 
6th topic
바이오 르호봇 중국 레이 NRDO 엘리베이터 셀러 생산 회장 대표 국내 직스 사업 플러스 해외 초음파 아시아 진단 수소차 미디어 


In [14]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:5]:
        print(words_list[index][1]['title'])        
    print("\n", "="*80)

0th topic
KB운용, 자사펀드로 자산배분 '재간접상품' 낸다           공모펀드로만 포트폴리오 구성, 판매사와 협의 단계…조재민 대표 '아이디어'
키움PE, 1호 블라인드펀드 소진 '숨고르기'           1년만에 40% 투자 집행…관리 집중
성장금융·KB금융, 사회적 기업 투자 나선다           250억 사회투자펀드 조성, 이르면 이달 위탁운용사 선정
마젤란기술투자, 300억 '엔젤세컨더리' GP로 선정           6번째 블라인드펀드 'MTI-엔젤세컨더리' 결성 예정, 4차산업 공략
스틱, 팬아시아4차산업펀드 1년만에 절반 소진           중국 조이비오 투자에 SK그룹과 함께 참여

1th topic
[아시아나항공 M&A]애경 vs 현대산업-미래대우 vs KCGI '3파전'오후 2시 마감…기존 원매자 세곳 응찰
[아시아나항공 M&A]예비입찰 함구령…GS그룹 참여했나 안했나정보차단 총력…입찰 이후에도 참여 가능성 열려있어
[아시아나항공 M&A]HDC현대산업개발, 유력 인수 후보로 깜짝 등장현금유동성 풍부…예비입찰 참여 여부 촉각
[아시아나항공 M&A]SK·한화, 결국 불참…그래도 여전한 잠재후보군흥행실패에 베팅, 예비입찰 불참해도 본입찰 참여 가능
[아시아나항공 M&A]항공업 미래 불투명한데…인수전 뛰어드는 이유는기존 사업과 시너지·다각화 기대, 왕성한 현금창출력도 매력

2th topic
건설업 채권발행 활발…발행사·투자자 '윈윈'           [Market Watch]신용등급 'AA~BBB0'까지 회사채 발행…저금리 조달효과 만끽
SK에너지, 공모채 추진…15년물 도전할까           3000억 규모 타진…대표주관사 KB증권 유력
SK㈜, 공모채 발행 추진…만기구조 고심           2500억 안팎…올해 세번째 조달, 연타석 오버부킹 기대
사모채 등급 평정 증가…체질개선 이루나           35곳 발행사 신용도 보유…연기금, 캐피탈 등 심사 강화 영향
홀로 선 'SK증권', 만족스런 공모채 여건

## Visualization

In [15]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지 (hard clustering)


W shape: (200, 7)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 200 samples in 0.000s...
[t-SNE] Computed neighbors for 200 samples in 0.000s...
[t-SNE] Computed conditional probabilities for sample 200 / 200
[t-SNE] Mean sigma: 0.069324
[t-SNE] KL divergence after 250 iterations with early exaggeration: 59.818451
[t-SNE] KL divergence after 900 iterations: 0.261740


In [16]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [17]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})



p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [18]:
#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요
from sklearn.preprocessing import normalize

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob


In [19]:
def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

In [20]:
doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)

In [21]:
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn


print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 20, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
show(prepared_data)

(7, 6486)
6486


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8894/    [Ctrl-C to exit]


127.0.0.1 - - [05/Sep/2019 14:24:20] "GET / HTTP/1.1" 200 -



stopping Server...


## Simple search engine

In [22]:
query = '바이오'
query_index = word2id[query]  # 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
#print('논제로_독', nonzero_doc);
ranks = query_vec.toarray().flatten().argsort()[::-1]
#print('랭크', ranks);



for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])


이지바이오, 마니커F&G 끝으로 계열사 IPO 쉼표           2016년부터 계열사 4곳 코스닥 입성…재무건전성 개선효과 https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908290100053110003279&lcode=00&page=8&svccode=00
[바이오 CMO의 재발견]넥타, 제조이슈에 시총 급락…CMC 재조명④BMS와 4조 딜 체결하고 임상시료 품질 확보 못해…CDMO 활용이 성패 관리 '키' https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908280100048590002997&lcode=00&page=4&svccode=00
브릿지바이오, 기술성평가 '삼수' 성공…IPO 박차           평가기관 두곳에서 모두 'A'…내달 초 코스닥 예심 청구 https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908290100052800003262&lcode=00&page=14&svccode=00
'1.5조 기술이전' 브릿지바이오, IPO 스타트           내주 상장 예비심사 청구…기술성평가도 재신청 https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908280100049160003029&lcode=00&page=17&svccode=00
인트론바이오, 300억 포기…미래 불확실성에 베팅           L/O 초기 마일스톤 포기…회사 측 "파마반트1 자금 문제 아냐, 4년뒤 상품화 주력" https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908300100055710003441&lcode=00&page=5&svccode=00
브릿지바이오, L/O 450억 유입…흑자 바이오 된다           계약금 및 단기마일스톤 달성…조만간 두 차례